# RSA

In [10]:
import sys, threading


sys.setrecursionlimit(10**7)
threading.stack_size(2**27)


def ConvertToInt(message_str):
    res = 0
    for i in range(len(message_str)):
        res = res * 256 + ord(message_str[i])
    return res

def ConvertToStr(n):
    res = ""
    while n > 0:
        res += chr(n % 256)
        n //= 256
    return res[::-1]

def PowMod(a, n, mod):
    if n == 0:
        return 1 % mod
    elif n == 1:
        return a % mod
    else:
        b = PowMod(a, n // 2, mod)
        b = b * b % mod
        if n % 2 == 0:
            return b
        else:
            return b * a % mod

def ExtendedEuclid(a, b):
    if b == 0:
        return (1, 0)
    (x, y) = ExtendedEuclid(b, a % b)
    k = a // b
    return (y, x - k * y)

def InvertModulo(a, n):
    (b, x) = ExtendedEuclid(a, n)
    if b < 0:
        b = (b % n + n) % n
    return b

In [11]:
# 1
def Encrypt(message, modulo, exponent):
    return PowMod(ConvertToInt(message), exponent, modulo)

In [13]:
# 2
def Decrypt(ciphertext, p, q, exponent):
    phi = (p-1)*(q-1)
    d = InvertModulo(exponent, phi)
    return ConvertToStr(PowMod(ciphertext, d, p * q))
  
a = 3
b = 7
c = InvertModulo(a, b)
print(c)

p = 1000000007
q = 1000000009
exponent = 23917
modulo = p * q

ciphertext = Encrypt("attack", modulo, exponent)
message = Decrypt(ciphertext, p, q, exponent)
print(message)

5
attack


In [ ]:
# 3
def DecipherSimple(ciphertext, modulo, exponent, potential_messages):
    for message in potential_messages:
        if ciphertext == Encrypt(message, modulo, exponent):
            return message
    return "don't know"

modulo = 101
exponent = 12
ciphertext = Encrypt("attack", modulo, exponent)
print(ciphertext)
print(DecipherSimple(ciphertext, modulo, exponent, ["attack", "don't attack", "wait"]))

In [20]:
# 4
def DecipherSmallPrime(ciphertext, modulo, exponent):
    small_prime = 2
    while small_prime <= 1000000:
        if modulo % small_prime == 0:
            big_prime = modulo // small_prime
            m = Decrypt(ciphertext, small_prime, big_prime, exponent)
            return m
        small_prime += 1
    return "don't know"

#modulo = 101 * 18298970732541109011012304219376080251334480295537316123696052970419466495220522723330315111017831737980079504337868198011077274303193766040393009648852841770668239779097280026631944319501437547002412556176186750790476901358334138818777298389724049250700606462316428106882097210008142941838672676714188593227684360287806974345181893018133710957167334490627178666071809992955566020058374505477745993383434501768887090900283569055646901291270870833498474402084748161755197005050874785474707550376333429671113753137201128897550014524209754619355308207537703754006699795711188492048286436285518105948050401762394690148387
#exponent = 239
#ciphertext = Encrypt("attack", modulo, exponent)
#print(DecipherSmallPrime(ciphertext, modulo, exponent))
ciphertext = 2275574988111277110437311214647848935000427550724521671610776681372035260669927879546598212402059626568529572296378786125308417822064565716822176055495534704248368924219660507214955218285229437531331022539446136060045737304497347919927268903882567968402406701512120140117419155759639779776379512631504909857

modulo = 4517997441484302732739462818197633745340869919448215356148620255867108411028738486895653782573554891335084161684285442351914740415677262295900238601126392055761648706990593425725625329173683080677363007706828763867611649178136159927562319810830366004181626803874119100160889961902072311468445073289996506931

exponent = 1927740953777951565602110467665746890410015037617105427402060961409149865403375973001399584612131360178672080227818127622463053259278787951537329431243

decrypt_third_puzzle = DecipherSmallPrime(ciphertext, modulo, exponent)
print(decrypt_third_puzzle)

https://docs.google.com/document/d/1kubEo4ODZ_1YZl8Hvi9_


In [22]:
# 5
def IntSqrt(n):
    low = 1
    high = n
    iterations = 0
    while low < high and iterations < 5000:
        iterations += 1
        mid = (low + high + 1) // 2
        if mid * mid <= n:
            low = mid
        else:
            high = mid - 1
    return low

def DecipherSmallDiff(ciphertext, modulo, exponent):
    sqrt_modulo = IntSqrt(modulo)
    small_prime = sqrt_modulo
    while small_prime > sqrt_modulo - 5000:
        if modulo % small_prime == 0:
            big_prime = modulo // small_prime 
            m = Decrypt(ciphertext, small_prime, big_prime, exponent)
            return m
        small_prime-=1
    return "don't know"
  
p = 1000000007
q = 1000000009
n = p * q
e = 239
ciphertext = Encrypt("attack", n, e)
message = DecipherSmallDiff(ciphertext, n, e)
print(message)

attack


In [25]:
# 6
def GCD(a, b):
    if b == 0:
        return a
    return GCD(b, a % b)

def DecipherCommonDivisor(first_ciphertext, first_modulo, first_exponent, second_ciphertext, second_modulo, second_exponent):
  # Fix this implementation to correctly decipher both messages in case
  # first_modulo and second_modulo share a prime factor, and return
  # a pair (first_message, second_message). The implementation below won't work
  # if the common_prime is bigger than 1000000.
    common_prime = GCD(first_modulo, second_modulo)
    q1 = first_modulo // common_prime
    q2 = second_modulo // common_prime
    m1 = Decrypt(first_ciphertext, common_prime, q1, first_exponent)
    m2 = Decrypt(second_ciphertext, common_prime, q2, second_exponent)
    return (m1, m2)
  
# Example usage with common prime p and different second primes q1 and q2  
p = 101
q1 = 18298970732541109011012304219376080251334480295537316123696052970419466495220522723330315111017831737980079504337868198011077274303193766040393009648852841770668239779097280026631944319501437547002412556176186750790476901358334138818777298389724049250700606462316428106882097210008142941838672676714188593227684360287806974345181893018133710957167334490627178666071809992955566020058374505477745993383434501768887090900283569055646901291270870833498474402084748161755197005050874785474707550376333429671113753137201128897550014524209754619355308207537703754006699795711188492048286436285518105948050401762394690148387
q2 = 1000000007
first_modulo = p * q1
second_modulo = p * q2
first_exponent = 239
second_exponent = 17
first_ciphertext = Encrypt("Baby", first_modulo, first_exponent)
second_ciphertext = Encrypt("Long", second_modulo, second_exponent)
print(DecipherCommonDivisor(first_ciphertext, first_modulo, first_exponent, second_ciphertext, second_modulo, second_exponent))

('Baby', 'Long')


In [27]:
# 7
def ChineseRemainderTheorem(n1, r1, n2, r2):
    (x, y) = ExtendedEuclid(n1, n2)
    return ((r2 * x * n1 + r1 * y * n2) % (n1 * n2) + (n1 * n2)) % (n1 * n2)

def DecipherHastad(first_ciphertext, first_modulo, second_ciphertext, second_modulo):
    r = ChineseRemainderTheorem(first_modulo, first_ciphertext, second_modulo, second_ciphertext)
    return ConvertToStr(IntSqrt(r))
  
p1 = 790383132652258876190399065097
q1 = 662503581792812531719955475509
p2 = 656917682542437675078478868539
q2 = 1263581691331332127259083713503
n1 = p1 * q1
n2 = p2 * q2
ciphertext1 = Encrypt("attack", n1, 2)
ciphertext2 = Encrypt("attack", n2, 2)
message = DecipherHastad(ciphertext1, n1, ciphertext2, n2)
print(message)

attack


In [3]:
"ACaucusRaceandaLongTale".isalpha()

True